# Aortic Dissection Auto-segmentation for Computed Tomography (CT) 

This notebook is prepared for step-by-step description of auto-segmentation from the CT images of aortic dissection patients underwent TEVAR in Keck medical center.

In [1]:
# Import Packages

import numpy as np
import pandas as pd
import nibabel as nib
import matplotlib.pyplot as plt
import json
import cv2
import imageio
import os, glob
from PIL import Image

import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import backend as K 
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.preprocessing.image import ImageDataGenerator 


import seaborn as sns
import umetrics

In [2]:
#Delete the extra first!

In [3]:
# Define constants
SEED = 909
BATCH_SIZE_TRAIN = 32
BATCH_SIZE_TEST = 32

IMAGE_HEIGHT = 256
IMAGE_WIDTH = 256
IMG_SIZE = (IMAGE_HEIGHT, IMAGE_WIDTH)

data_dir = 'slices/'
data_dir_train = os.path.join(data_dir, 'training')
# The images should be stored under: "data/slices/training/img/img"
data_dir_train_image = os.path.join(data_dir_train, 'img')
# The images should be stored under: "data/slices/training/mask/img"
data_dir_train_mask = os.path.join(data_dir_train, 'mask')

save_img_path = os.path.join(data_dir_train, 'img/img')
save_mask_path = os.path.join(data_dir_train, 'mask/img')

data_dir_test = os.path.join(data_dir, 'test')
# The images should be stored under: "data/slices/test/img/img"
data_dir_test_image = os.path.join(data_dir_test, 'img')
# The images should be stored under: "data/slices/test/mask/img"
data_dir_test_mask = os.path.join(data_dir_test, 'mask')

def create_segmentation_generator_train(img_path, msk_path, BATCH_SIZE):
    data_gen_args = dict(rescale=1.0,
                         #featurewise_center=True,
                         #featurewise_std_normalization=True,
                         rotation_range=45,
                         width_shift_range=0.2,
                         height_shift_range=0.2,
                         zoom_range=0.3,
                         fill_mode="wrap",
                        )
    datagen = ImageDataGenerator(**data_gen_args)
    
    img_generator = datagen.flow_from_directory(img_path, target_size=IMG_SIZE, class_mode=None, color_mode='grayscale', batch_size=1, seed=SEED, save_to_dir=save_img_path, save_prefix="aug", save_format="png")
    total_image = 4181
    aug_ratio = 5
    i = 0
    for save_image in img_generator:
        if (i == total_image*aug_ratio-1):
            break
        i += 1
    
    msk_generator = datagen.flow_from_directory(msk_path, target_size=IMG_SIZE, class_mode=None, color_mode='grayscale', batch_size=1, seed=SEED, save_to_dir=save_mask_path, save_prefix="aug", save_format="png")
    i = 0
    for save_image in msk_generator:
        if (i == total_image*aug_ratio-1):
            break
        i += 1
    
    return (pair for pair in zip(img_generator, msk_generator))

#Apply the generator functions on the training and test data
train_generator = create_segmentation_generator_train(data_dir_train_image, data_dir_train_mask, 1)

Found 4181 images belonging to 1 classes.
Found 4181 images belonging to 1 classes.


In [4]:
#End of the Code